In [ ]:
%load_ext autoreload
%autoreload 2

from __future__ import annotations

import os
import sys
import time
import numpy as np
import uproot
import math
import itertools


from scipy.optimize import Bounds, minimize


simDir = os.getcwd()
analysisDir = os.path.join(simDir, '..', 'Analysis')
sys.path.append(analysisDir)

from simulationClass import FIMS_Simulation
from runDataClass import runData

In [ ]:
def scanParam(parameters, startValues, stopValues, numSteps):
    """
    Executes a parameter scan by running the simulation multiple times with
    varying combinations of specified parameters.

    Executes an N-dimensional linear scan with every combination of parameters.
    If a single simulation fails, its run number is set to -1 and the scan continues.

    Args:
        parameters (list): List of strings for parameters to be scanned.
        startValues (list): List of numbers; Starting value of each parameter.
        stopValues (list): List of numbers; Ending value of each parameter.
        numSteps (list): List of ints; Number of steps within each parameter's range.

    Returns:
        list: List of the run numbers of the executed simulations.
              Empty if the parameters are not recognized.
    """

    if not (len(parameters) == len(startValues) == len(stopValues) == len(numSteps)):
        raise ValueError('Inputs must have the same length.')

    simulation = FIMS_Simulation()
    allParams = simulation.defaultParam()

    for inParam in parameters:
        if inParam not in allParams:
            print(f"Parameter '{inParam}' not able to be varied.")
            return []
    
    #Find range for each parameter
    allRanges = [
        np.linspace(start, stop, step) for start, stop, step in zip(startValues, stopValues, numSteps)
    ]

    print(f'Beginning scan through {parameters}. {math.prod(numSteps)} runs.')

    runNos = []
    for scanParams in itertools.product(*allRanges):

        #Reset parameters to defaults
        simulation.resetParam(verbose=False)
        
        #Apply current parameter combination
        for i, inParam in enumerate(parameters):
            simulation.param[inParam] = scanParams[i]

        runNo = simulation.runSimulation()
        if runNo == -1:
            print(f'Simulation failed with settings: {simulation}')
        runNos.append(runNo)

    print(f'Done scan through {parameters}. Final run number: {runNos[-1]}')
    simulation.resetParam()
        
    return runNos

In [ ]:
def scanValues(parameterDefinitions):
    """
    Executes a parameter scan by running the simulation multiple times with
    varying combinations of specified parameters.

    Executes an N-dimensional linear scan with every combination of parameters.
    If a single simulation fails, its run number is set to -1 and the scan continues.

    Args:
        parameterDefinitions (list): List of lists with the following form: 
                                     ['parameter', minValue, maxValue, stepValue]
                                     'parameter' - String of the parameter name.
                                      startValue - Number for the starting parameter value.
                                      stopValue - Number for the ending parameter value.
                                      stepValue - Integer number of steps through range.

    Returns:
        list: List of the run numbers of the executed simulations.
              Empty if the parameters are not recognized.
    """
    # Use a list comprehension to validate the input first
    if not all(len(inDefinition) == 4 for inDefinition in parameterDefinitions):
        raise ValueError("Input list elements must have the form: ['parameter', startValue, stopValue, stepValue]")
    
    # Unpack the transposed lists
    parameters, startValues, stopValues, numSteps = zip(*parameterDefinitions)
    # Convert the resulting tuples to lists
    parameters = list(parameters)
    startValues = list(startValues)
    stopValues = list(stopValues)
    numSteps = list(numSteps)

    #run the scan
    startTime = time.monotonic()
    runNos = scanParam(parameters, startValues, stopValues, numSteps)
    endTime = time.monotonic()
    print(f'Scan completed in ~{endTime-startTime:.1f}s.')
        
    return runNos

In [ ]:
# Default run of a simulation.
## Determines a start-to-finish execution time.
## Individual program times are from log files.
if __name__ == '__main__':
    defaultSim = FIMS_Simulation()
    startTime = time.monotonic()
    defaultSim.runSimulation()
    endTime = time.monotonic()
    print(f'Default run takes ~{endTime-startTime:.1f}s.')

    '''
    Last time Tanner ran this test (10/27/25):
    Total time: ~ 2147 s
        Gmsh: ~286 s
        ElmerGrid: ~2 s   
        ElmerSolver: ~ 1527 s
        ElmerWeighting: ~161 s
        Garfield: ~172 s
    '''
    """
    Last time James ran this test:
    Total time: ~2,827 s
       Gmsh: ~204 s
       ElmerGrid: ~1 s   
       ElmerSolver: ~878 s
       ElmerWeighting: ~50 s
       Garfield: ~1,693 s
    """

In [ ]:
fieldForEff = FIMS_Simulation()
startTime = time.monotonic()
fieldForEff.findFieldForEfficiency()
endTime = time.monotonic()
print(f'Finding default field takes ~{endTime-startTime:.1f}s.')

In [ ]:
efficiencyTest = [['fieldRatio', 50, 100, 6]]
scanValues(efficiencyTest)